In [423]:
import numpy as np
import matplotlib.pyplot as plt

In [424]:
words = ["boy", "bat"]
print(len(words))

2


# Get data Ready

In [587]:
def encoder(words, inputs=False):
    encoded = []
    for word in words:
        ohe = []
        for i in range(27):
            ohe.append(0)

        if inputs:
            ohe[ord(word[0]) - 97] = 1
        else:
            ohe[ord(word[1]) - 97] = 1
        encoded.append(ohe)
    
    return np.array(encoded)

In [588]:
inputs = np.float32(encoder(words, inputs=True))
labels = np.float32(encoder(words, inputs=False))

In [589]:
print(inputs.shape)
print(labels.shape)

(2, 27)
(2, 27)


In [415]:
def decoder(a):
    ans = []
    for res in a:
        ans.append(chr(res + 97))
    return np.array(ans)

In [22]:
def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

In [23]:
def tanh_derivative(x):
    return 1 - tanh(x) ** 2

In [162]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [6]:
def encoder_v2(char, end=False):
    ohe = []
    for i in range(27):
        ohe.append(0)
    if end:
        ohe[-1] = 1
    else:
        ohe[ord(char)-97] = 1
    return ohe

In [18]:
for word in words:
    for i in range(len(word)):
        input = encoder_v2(word[i], False)
        if i == len(word)-1:
            output = encoder_v2(word[i], True)
        else:
            output = encoder_v2(word[i+1], False)

        print(word[i] + ":")
        print(input)
        print(output)
        print()

b:
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

o:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

y:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

b:
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

a:
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

t:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Scratch Work

## Initialization

In [591]:
wx = np.random.rand(27, 8)
wh = np.random.rand(8, 8)
wy = np.random.rand(8, 27)

bx = np.zeros((8,))
bh = np.zeros((8,))
by = np.zeros((27,))

h = np.zeros((inputs.shape[0], 8))

print(wx.shape)
print(wh.shape)
print(wy.shape)

print(bx.shape)
print(bh.shape)
print(by.shape)

print(h.shape)

(27, 8)
(8, 8)
(8, 27)
(8,)
(8,)
(27,)
(2, 8)


## Initialization V2

In [592]:
lr = 1e-3

In [603]:
input_array = []
output_array = []
a2_array = []
z2_array = []
a1_array = []
z1_array = []

da2_array = []
da1_array = []

In [604]:
dwy = np.zeros((8, 27))
dwa = np.zeros((8, 8))
dwx = np.zeros((27, 8))

dby = np.zeros((27,))
dba = np.zeros((8,))
dbx = np.zeros((8,))

In [605]:
for word in words:
    input_array.append([])
    output_array.append([])
    a2_array.append([])
    z2_array.append([])
    a1_array.append([])
    z1_array.append([])
    
    da2_array.append([])
    da1_array.append([])

In [606]:
j = 0
for word in words:
    a1_array[j].append(np.zeros((inputs.shape[0], 8)))
    j+=1

## Forward

In [489]:
z1 = np.dot(inputs, wx) + bx + np.dot(h, wh) + bh
a1 = tanh(z1)

print(z1.shape)
print(a1.shape)

(1, 8)
(1, 8)


In [490]:
z2 = np.dot(a1, wy) + by
a2 = tanh(z2)

print(z2.shape)
print(a2.shape)

(1, 27)
(1, 27)


## Forward V2

In [ ]:
j = 0
for word in words:
    for i in range(len(word)):
        input = encoder_v2(word[i], False)
        if i == len(word)-1:
            output = encoder_v2(word[i], True)
        else:
            output = encoder_v2(word[i+1], False)

        print(word[i] + ":")
        print(input)
        print(output)
        print()


        input = input.reshape(1, 27)
        output = output.reshape(1, 27)


        input_array[j].append(input)
        output_array[j].append(output)
        
        #Forward
        z1 = np.dot(input, wx) + bx + np.dot(a1_array[j][-1], wh) + bh
        a1 = tanh(z1)

        z1_array[j].append(z1)
        a1_array[j].append(a1)

        z2 = np.dot(a1, wy) + by
        a2 = tanh(z2)

        z2_array[j].append(z2)
        a2_array[j].append(a2)


    j+=1

In [27]:
test = []
for ti in range(len(words[0])):
    test.append([])
    for batch in range(len(words)):
        test[ti].append([])



for i in range(len(words[0])):
    for j in range(len(words)):
        input = encoder_v2(words[j][i], False)
        if i == len(words[0])-1:
            output = encoder_v2(words[j][i], True)
        else:
            output = encoder_v2(words[j][i+1], False)

        # print(words[j][i] + ":")
        # print(input)
        # print(output)
        # print()

        test[i][j] = input
# print(test)
test = np.array(test)
print(test.shape)

(3, 2, 27)


In [135]:
rnn = RNN()
rnn.preprocess(words)
print()
rnn.forward(words)
rnn.backward(words, 1e-3)

print()
rnn.loss()

(3, 2, 27)
(3, 2, 27)




0.8534494519137084

## Backward V2

In [611]:
input_array[0][2].shape

(1, 27)

In [598]:
j = 0
for word in words:
    for i in range(len(word)-1, -1, -1):
        print(i)

        dL_da2 = 2 * (a2_array[j][i] - labels[j])
        
        dL_da1 = dL_da2 * tanh_derivative(z2[j][i])
        dL_da1 = np.dot(dL_da1, wy.T)
        
        print(dL_da2.shape)
        print(dL_da1.shape)

        # Weights
        dL_dwy = dL_da2 * tanh_derivative(z2[j][i])
        dL_dwy = np.dot(a1_array[j][i+1].T, dL_dwy)

        print(dL_dwy.shape)

        dL_dwa = dL_da1 * tanh_derivative(z1[j][i])
        dL_dwa = np.dot(a1_array[j][i].T, dL_dwa)

        print(dL_dwa.shape)

        dL_dwx = dL_da1 * tanh_derivative(z1[j][i])
        dL_dwx = np.dot(input_array[j][i].T, dL_dwx)

        print(dL_dwx.shape)

        #Biases
        dL_dby = np.mean(dL_da2 * tanh_derivative(z2[j][i]), axis=0)
        print(dL_dby.shape)

        dL_dba = np.mean(dL_da1 * tanh_derivative(z1[j][i]), axis=0)
        print(dL_dba.shape)

        dL_dbx = np.mean(dL_da1 * tanh_derivative(z1[j][i]), axis=0)
        print(dL_dbx.shape)

        #Accumulate Gradients
        dwy+=dL_dwy
        dwa+=dL_dwa
        dwx+=dL_dwx

        dby+=dL_dby
        dba+=dL_dba
        dbx+=dL_dbx

    j+=1

2
(2, 27)
(2, 8)
(8, 27)
(8, 8)


ValueError: ignored

In [585]:
print(np.mean(dwy), np.mean(dwa), np.mean(dwx), np.mean(dby), np.mean(dba), np.mean(dbx))

0.0042966157476157394 5.3938491032053164e-05 3.7400719949685634e-06 0.00461179552831038 0.0001009819438641512 0.0001009819438641512


## Backward

In [ ]:
loss = np.mean((a2 - labels) ** 2)
loss

0.8183981140471699

### Calculate Gradients

In [ ]:
dL_da2 = 2 * (a2 - labels)
da2_dz2 = tanh_derivative(z2)
dz2_dwy = a1.T

dwy = np.dot(dz2_dwy, dL_da2 * da2_dz2)
print(dwy.shape)

(8, 26)


In [ ]:
dz2_dby = 1

dby = np.mean(dL_da2 * da2_dz2 * dz2_dby, axis=0)
print(dby.shape)

(26,)


In [225]:
dz2_da1 = wy.T
da1_dz1 = tanh_derivative(z1)
dz1_dwh = h

dwh = np.dot((da1_dz1 * dz1_dwh).T, np.dot(dL_da2 * da2_dz2, dz2_da1))
print(dwh.shape)
print(np.mean(dwh))

(8, 8)
0.0


In [ ]:
dz1_dbh = 1

dbh = np.mean(np.dot(dL_da2 * da2_dz2, dz2_da1) * (da1_dz1 * dz1_dwh), axis=0)
print(dbh.shape)

In [ ]:
dz1_dwx = inputs.T

dwx = np.dot(dz1_dwx, np.dot(dL_da2 * da2_dz2, dz2_da1) * da1_dz1)
print(dwx.shape)

(26, 8)


In [ ]:
dbx = dbh
print(dbx.shape)

(8,)


### Apply Gradients

In [ ]:
wy -= dwy * lr
wh -= dwh * lr
wx -= dwx * lr

by -= dby * lr
bh -= dbh * lr
bx -= dbx * lr

# Put it into a function

In [270]:
def train(inputs, labels, numb_epochs=5, lr=1e-3):
    wx = np.random.rand(26, 8)
    wh = np.random.rand(8, 8)
    wy = np.random.rand(8, 26)

    bx = np.zeros((8,))
    bh = np.zeros((8,))
    by = np.zeros((26,))

    h = np.zeros((inputs.shape[0], 8))

    for epoch in range(1, numb_epochs+1):
        #Forward
        z1 = np.dot(inputs, wx) + bx + np.dot(h, wh) + bh
        a1 = tanh(z1)

        z2 = np.dot(a1, wy) + by
        a2 = tanh(z2)

        #Backward
        loss = np.mean((a2 - labels) ** 2)
        print(loss)

        ## Calculate Gradients
        dL_da2 = 2 * (a2 - labels)
        da2_dz2 = tanh_derivative(z2)
        dz2_dwy = a1.T
        dwy = np.dot(dz2_dwy, dL_da2 * da2_dz2)

        dz2_dby = 1
        dby = np.mean(dL_da2 * da2_dz2 * dz2_dby, axis=0)

        dz2_da1 = wy.T
        da1_dz1 = tanh_derivative(z1)
        dz1_dwh = h
        dwh = np.dot(dz1_dwh.T, np.dot(dL_da2 * da2_dz2, dz2_da1) * da1_dz1)

        dz1_dbh = 1
        dbh = np.mean(np.dot(dL_da2 * da2_dz2, dz2_da1) * (da1_dz1 * dz1_dwh), axis=0)

        dz1_dwx = inputs.T
        dwx = np.dot(dz1_dwx, np.dot(dL_da2 * da2_dz2, dz2_da1) * da1_dz1)

        dbx = dbh

        ## Apply Gradients
        wy -= dwy * lr
        wh -= dwh * lr
        wx -= dwx * lr

        by -= dby * lr
        bh -= dbh * lr
        bx -= dbx * lr

    return wx, wh, wy, bx, bh, by

In [271]:
wx, wh, wy, bx, bh, by = train(inputs, labels, 1000, 1e-3)

0.8572647741254251
0.8551619084501413
0.8529877184375853
0.850739101958367
0.8484128108851199
0.8460054457136118
0.8435134503974843
0.8409331075040288
0.838260533820687
0.8354916765677867
0.8326223104027846
0.8296480354352617
0.8265642765103228
0.8233662840609552
0.8200491368770793
0.8166077471899538
0.8130368685241811
0.8093311068240804
0.80548493541398
0.8014927143992283
0.7973487151512146
0.7930471505385254
0.7885822115587798
0.7839481109809157
0.7791391345131841
0.7741496998538699
0.7689744237453852
0.7636081968246952
0.7580462656344178
0.7522843206267407
0.7463185883650473
0.7401459254302901
0.733763910815901
0.7271709329161175
0.7203662666733847
0.7133501361687965
0.7061237580459246
0.6986893617785431
0.6910501840214929
0.6832104361527236
0.6751752465572771
0.6669505820289975
0.6585431555450999
0.6499603301552433
0.6412100303072907
0.6323006721164819
0.623241122520403
0.6140406938487125
0.6047091753124171
0.5952568968839727
0.5856948149030673
0.5760346035728003
0.5662887333333424

In [247]:
z1 = np.dot(inputs, wx) + bx + np.dot(h, wh) + bh
a1 = tanh(z1)

z2 = np.dot(a1, wy) + by
a2 = softmax(z2)

In [248]:
print(decoder(np.argmax(a2, axis=1)))

['t' 'n' 'i' 'e' 'o' 'e' 'e' 'o' 'o' 'o']


# Class Method

In [51]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [52]:
def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x))

In [425]:
class RNN:
    def __init__(self):
        self.h_neurons = 8
        self.out_neurons = 27
        self.reset()
        self.inputs_array = []
        self.outputs_array = []

        self.wx = np.random.rand(27, 8)
        self.wa = np.random.rand(8, 8)
        self.wy = np.random.rand(8, 27)

        self.bx = np.zeros((8,))
        self.ba = np.zeros((8,))
        self.by = np.zeros((27,))

        self.best_loss = 1e7
        self.best_wx = self.wx
        self.best_wa = self.wa
        self.best_wy = self.wy
        
        self.best_bx = self.bx
        self.best_ba = self.ba
        self.best_by = self.by

    def preprocess(self, words):
        inputs_array = []
        outputs_array = []
        for ti in range(len(words[0])):
            inputs_array.append([])
            outputs_array.append([])
            for batch in range(len(words)):
                inputs_array[ti].append([])
                outputs_array[ti].append([])
        
        for ti in range(len(words[0])):
            for batch in range(len(words)):
                input = encoder_v2(words[batch][ti], False)
                if ti == len(words[0])-1:
                    output = encoder_v2(words[batch][ti], True)
                else:
                    output = encoder_v2(words[batch][ti+1], False)

                inputs_array[ti][batch] = input
                outputs_array[ti][batch] = output

        inputs_array = np.array(inputs_array)
        outputs_array = np.array(outputs_array)

        self.inputs_array = inputs_array
        self.outputs_array = outputs_array

    def reset(self, time_steps=1, batch_size=1):
        self.z1_array = np.zeros((time_steps, batch_size, self.h_neurons))
        self.a1_array = np.zeros((time_steps+1, batch_size, self.h_neurons))
        self.z2_array = np.zeros((time_steps, batch_size, self.out_neurons))
        self.a2_array = np.zeros((time_steps, batch_size, self.out_neurons))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    def tanh_derivative(self, x):
        return 1 - self.tanh(x) ** 2

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def forward(self, words):
        self.reset(time_steps=len(words[0]), batch_size=len(words))
        
        for ti in range(len(words[0])):
            input = self.inputs_array[ti]
            
            #Forward
            z1 = np.dot(input, self.wx) + self.bx + np.dot(self.a1_array[ti], self.wa) + self.ba
            a1 = self.tanh(z1)

            self.z1_array[ti] = z1
            self.a1_array[ti+1] = a1

            z2 = np.dot(a1, self.wy) + self.by
            a2 = self.sigmoid(z2)

            self.z2_array[ti] = z2
            self.a2_array[ti] = a2

    def loss(self):
        loss = np.mean((self.a2_array - self.outputs_array) ** 2)
        if loss < self.best_loss:
            self.best_wx = self.wx
            self.best_wa = self.wa
            self.best_wy = self.wy
            
            self.best_bx = self.bx
            self.best_ba = self.ba
            self.best_by = self.by

        return loss

    def backward(self, words, lr=1e-3):
        dwy = 0
        dwa = 0
        dwx = 0

        dby = 0
        dba = 0
        dbx = 0

        for ti in range(len(words[0])-1, -1, -1):

            dL_da2 = 2 * (self.a2_array[ti] - self.outputs_array[ti])
            # print(dL_da2.shape)

            dL_da1 = dL_da2 * self.sigmoid_derivative(self.z2_array[ti])
            dL_da1 = np.dot(dL_da1, self.wy.T)
            # print(dL_da1.shape)

            # Weights
            dL_dwy = dL_da2 * self.sigmoid_derivative(self.z2_array[ti])
            dL_dwy = np.dot(self.a1_array[ti+1].T, dL_dwy)
            # print(dL_dwy.shape)

            dL_dwa = dL_da1 * self.tanh_derivative(self.z1_array[ti])
            dL_dwa = np.dot(self.a1_array[ti].T, dL_dwa)
            # print(dL_dwa.shape)

            dL_dwx = dL_da1 * self.tanh_derivative(self.z1_array[ti])
            dL_dwx = np.dot(self.inputs_array[ti].T, dL_dwx)
            # print(dL_dwx.shape)

            #Biases
            dL_dby = np.mean(dL_da2 * self.sigmoid_derivative(self.z2_array[ti]), axis=0)
            # print(dL_dby.shape)

            dL_dba = np.mean(dL_da1 * self.tanh_derivative(self.z1_array[ti]), axis=0)
            # print(dL_dba.shape)

            dL_dbx = np.mean(dL_da1 * self.tanh_derivative(self.z1_array[ti]), axis=0)
            # print(dL_dbx.shape)


            #Accumulate Gradients
            dwy += dL_dwy
            dwa += dL_dwa
            dwx += dL_dwx

            dby += dL_dby
            dba += dL_dba
            dbx += dL_dbx

        self.wy -= dwy * lr
        self.wa -= dwa * lr
        self.wx -= dwx * lr

        self.by -= dby * lr
        self.ba -= dba * lr
        self.bx -= dbx * lr

In [426]:
rnn = RNN()
rnn.preprocess(words)
rnn.forward(words)
rnn.backward(words, 1e-3)

rnn.loss()

0.8304052464569587

In [428]:
words = []
with open("five_letter.txt", "r") as f:
    for line in f:
        words.extend(line.split())

In [429]:
len(words)

5757

In [430]:
rnn = RNN()
rnn.preprocess(words)

In [431]:
def train(words, model, numb_epochs):
    for epoch in range(1, numb_epochs+1):
        model.forward(words)
        model.backward(words, 1e-5)

        print(epoch, model.loss())


In [433]:
train(words, rnn, 10000)

1 0.5673363312419775
2 0.4947261729461323
3 0.4478246156433707
4 0.4039456306088321
5 0.3595000961201335
6 0.3117948181748246
7 0.263926934320091
8 0.22324153522064125
9 0.19027548639418673
10 0.1646893052247709
11 0.14528453736438723
12 0.13044808232349422
13 0.11900698614371652
14 0.10965360410144454
15 0.10131225474562905
16 0.09377652526404891
17 0.08753656765260007
18 0.08282139333946816
19 0.07929258199577024
20 0.07653263763984787
21 0.0742671388551889
22 0.07233664994978035
23 0.07064648798611523
24 0.06913720871792876
25 0.0677693603371654
26 0.06651548860968866
27 0.06535573361524866
28 0.06427527028711182
29 0.06326274422242101
30 0.062309272574989756
31 0.061407781047420375
32 0.06055254922844479
33 0.05973888986140289
34 0.05896291699330989
35 0.058221374779593375
36 0.05751150871599872
37 0.056830967204020165
38 0.0561777252320574
39 0.05555002446751918
40 0.05494632572278602
41 0.05436527088792909
42 0.05380565220317128
43 0.053266387291733164
44 0.052746498764044636
45 

KeyboardInterrupt: ignored

In [434]:
wx, wa, wy, bx, ba, by = rnn.best_wx, rnn.best_wa, rnn.best_wy, rnn.best_bx, rnn.best_ba, rnn.best_by

In [435]:
inputs_array, outputs_array = rnn.inputs_array, rnn.outputs_array

In [436]:
def inference(words, inputs_array, outputs_array, wx, wa, wy, bx, ba, by):
    time_steps = len(words[0])
    batch_size = len(words)

    a1_array = np.zeros((time_steps+1, batch_size, 8))

    preds = []

    for ti in range(len(words[0])):
        input = inputs_array[ti]
        
        #Forward
        z1 = np.dot(input, wx) + bx + np.dot(a1_array[ti], wa) + ba
        a1 = tanh(z1)

        a1_array[ti+1] = a1

        z2 = np.dot(a1, wy) + by
        a2 = softmax(z2)
        preds.append(a2)

    return np.array(preds)

In [437]:
preds = inference(words, inputs_array, outputs_array, wx, wa, wy, bx, ba, by)

In [439]:
np.mean((preds - outputs_array) ** 2)

0.03702396378201403

In [440]:
preds.shape, outputs_array.shape

((5, 5757, 27), (5, 5757, 27))

In [ ]:
j = 0
for i in range(5):
    print("True:", words[j][i])
    print("Predicted:", chr(np.argmax(preds[i][j]) + 97))